# Explaining Data and Models

![](http://aix360.mybluemix.net/static/images/methods-choice.gif)

## What do you want to understand?

### Global explanation of data 

* [Health and Lifestyle Survey (CDC)](#CDC)
  * [Summarize Income Questionnaire using Prototypes](#summarize-questionnaire) (ProtoDash)

### Local explanation of a model

* [Health and Lifestyle Survey (CDC)](#CDC)
  * [Find Questionnaires that are most representative of Income](#find-questionnaire) (ProtoDash)
* [IRIS](#IRIS)
  * knn classification - [Explain a single prediction](#explain-single-shap) (SHAP)
* [Employee Retention](#retention)
  * [Explain a single prediction](#ted1) (TED)
  
### Global explanation of a model

* [IRIS](#IRIS)
  * [Explain all predictions](#explain-all-shap) (SHAP)
* [Employee Retention](#retention)
  * [Overall model accuracy](#ted2) (TED)

## Install and import packages

In [ ]:
!pip install numpy --upgrade
!pip install aix360

**When there are errors when running the below, restart kernel and run the cell below again**

In [ ]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')

import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import sklearn.datasets
import sklearn.ensemble
from sklearn import svm     
import time
np.random.seed(1)

from aix360.algorithms.protodash import ProtodashExplainer
from aix360.algorithms.ted.TED_Cartesian import TED_CartesianExplainer
from aix360.algorithms.shap import KernelExplainer
from aix360.datasets.cdc_dataset import CDCDataset
from aix360.datasets.ted_dataset import TEDDataset

import shap

<a class="anchor" id="CDC"></a>
# Understanding data - Health and Lifestyle Survey

The [NHANES CDC questionnaire datasets](https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Questionnaire&CycleBeginYear=2013) are surveys conducted by the organization involving thousands of civilians about their daily lives. There are 44 questionnaires that collect data about income, occupation, health, early childhood and many other behavioral and lifestyle aspects of individuals living in the US. 

### [Protodash: Fast Interpretable Prototype Selection](https://arxiv.org/abs/1707.01212)

* The method takes as input a datapoint (or group of datapoints) to be explained with respect to points in a training set belonging to the same feature space
* The method then tries to minimize the maximum mean discrepancy (MMD metric) between the datapoints we want to explain and a prespecified number of instances from the training set that it will select. It will try to select training instances that have the same distribution as the datapoints we want to explain
* The method has quality guarantees with it, returning importance weights for the chosen prototypical training instances indicative of how similar/representative they are

### Load data

This dataset is baked into aix360, which makes it easy to download:

In [ ]:
nhanes = CDCDataset()
nhanes_files = nhanes.get_csv_file_names()
(nhanesinfo, _, _) = nhanes._cdc_files_info()

Each column in the dataset corresponds to a question and each row are the answers given by a respondent to those questions. Both column names and answers by respondents are encoded. For example, 'SEQN' denotes the sequence number assigned to a respondent and 'IND235' corresponds to a question about monthly family income. In most cases a value of 1 implies "Yes" to the question, while a value of 2 implies "No." More details about the income questionaire and how questions and answers are encoded is [here](https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/INQ_H.htm).

In [ ]:
# replace encoded column names by the associated question text. 
df_inc = nhanes.get_csv_file('INQ_H.csv')
df_inc.columns[0]
dict_inc = {
'SEQN': 'Respondent sequence number', 
'INQ020': 'Income from wages/salaries',
'INQ012': 'Income from self employment',
'INQ030':'Income from Social Security or RR',
'INQ060':  'Income from other disability pension', 
'INQ080':  'Income from retirement/survivor pension',
'INQ090':  'Income from Supplemental Security Income',
'INQ132':  'Income from state/county cash assistance', 
'INQ140':  'Income from interest/dividends or rental', 
'INQ150':  'Income from other sources',
'IND235':  'Monthly family income',
'INDFMMPI':  'Family monthly poverty level index', 
'INDFMMPC':  'Family monthly poverty level category',
'INQ244':  'Family has savings more than $5000',
'IND247':  'Total savings/cash assets for the family'
}
qlist = []
for i in range(len(df_inc.columns)):
    qlist.append(dict_inc[df_inc.columns[i]])
df_inc.columns = qlist
print("Answers given by some respondents to the income questionnaire:")
df_inc.head(5).transpose()

In [ ]:
print("Number of respondents to Income questionnaire:", df_inc.shape[0])
print("Distribution of answers to \'monthly family income\' and \'Family savings\' questions:")

fig, axes = plt.subplots(1, 2, figsize=(10,5))
fig.subplots_adjust(wspace=0.5)
hist1 = df_inc['Monthly family income'].value_counts().plot(kind='bar', ax=axes[0])
hist2 = df_inc['Family has savings more than $5000'].value_counts().plot(kind='bar', ax=axes[1])
plt.show()

* The majority of individuals responded with a "12" for the question related to monthly family income, which means their income is above USD 8400 as explained [here](https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/INQ_H.htm#IND235)
* To the question of whether the family has savings of more than USD 5000, the majority of individuals responded with a "2", which means "No"

<a name="summarize-questionnaire"></a>
## Summarize Income Questionnaire using Prototypes

Consider a social scientist who would like to quickly obtain a summary report of this dataset in terms of types of people that span this dataset. Is it possible to summarize this dataset by looking at answers given by a few representative/prototypical respondents? 

The Protodash algorithm can be used to obtain a few prototypical respondents (about 10 in this example) that span the diverse set of individuals answering the income questionnaire making it easy to summarize the dataset.

In [ ]:
# convert pandas dataframe to numpy
data = df_inc.to_numpy()

#sort the rows by sequence numbers in 1st column 
idx = np.argsort(data[:, 0])  
data = data[idx, :]

# replace nan's (missing values) with 0's
original = data
original[np.isnan(original)] = 0

# delete 1st column (sequence numbers)
original = original[:, 1:]

print(original.shape)
original

In [ ]:
# one hot encode all features as they are categorical
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(original)

In [ ]:
print(onehot_encoded.shape)
onehot_encoded

### Why Protodash?

Prototypes represent the different segments in a dataset
* For example you might find that there are three categories of people: i) those that are high earners, ii) those that are middle class and iii) those that don't earn much or are unemployed and receive unemployment benefits
* Protodash will find these segments by pointing to specific individuals that lie in these categories
* From the objective function value of Protodash you are also able to say that three segments is the right number here as adding one more segment may not improve the objective value by much

The ['ProtodashExplainer'](v) provides exemplar-based explanations for summarizing datasets as well as explaining predictions.

* Input:  
    * Dataset to select prototypical explanations from: `onehot_encoded`
    * Dataset you want to explain: also `onehot_encoded`
    * Number of prototypes `m`
* Output:
* Indices of the selected prototypes `S`
* Importance weights associated with the selected prototypes `W`


In [ ]:
explainer = ProtodashExplainer()

(W, S, _) = explainer.explain(onehot_encoded, onehot_encoded, m=10) 

# sort the order of prototypes in set S
idx = np.argsort(S)
S = S[idx]
W = W[idx]

In [ ]:
S

In [ ]:
W

In [ ]:
# Display the prototypes along with their computed weights
inc_prototypes = df_inc.iloc[S, :].copy()
# Compute normalized importance weights for prototypes
inc_prototypes["Weights of Prototypes"] = np.around(W/np.sum(W), 2) 
inc_prototypes.transpose()

The 10 people shown above (i.e. 10 prototypes) are representative of the income questionnaire according to Protodash. 

* In the distribution plot for family finance related questions, there roughly were five times as many people not having savings in excess of USD 5000 compared with others. The prototypes also have a similar spread, which is reassuring
* For monthly family income, there is a more even spread over the more commonly occurring categories. This is a kind of spot check to see if our prototypes actually match the distribution of values in the dataset.
* Most people are employed (3rd question) and work for an organization earning through salary/wages (1st two questions)
* Most of them are also young (5th question) and fit to work (4th question)
* They don't seem to have much savings (last question)

<a name="find-questionnaire"></a>
## Find Questionnaires that are most representative of Income

How do the remaining 39 questionnaires represent or relate to income? The below will give an idea of which lifestyle factors are likely to affect income the most from prototypes for each of the questionnaires.

### Compute prototypes for all questionaires

This step uses Protodash explainer to compute 10 prototypes for each of the questionaires and saves these for further evaluation. (This takes a little while)

In [ ]:
# Iterate through all questionnaire datasets and find 10 prototypes for each.

prototypes = {}

for i in range(len(nhanes_files)):
    
    f = nhanes_files[i]
    
    print("processing ", f)
    
    # read data to pandas dataframe
    df = nhanes.get_csv_file(f)
    
    # convert data to numpy
    data = df.to_numpy()

    #sort the rows by sequence numbers in 1st column 
    idx = np.argsort(data[:, 0])
    data = data[idx, :]

    # replace nan's with 0's.
    original = data
    original[np.isnan(original)] = 0

    # delete 1st column (contains sequence numbers)
    original = original[:, 1:]

    # one hot encode all features as they are categorical
    onehot_encoder = OneHotEncoder(sparse=False)
    onehot_encoded = onehot_encoder.fit_transform(original)

    explainer = ProtodashExplainer()

    # call Protodash explainer
    # S contains indices of the selected prototypes
    # W contains importance weights associated with the selected prototypes 

    (W, S, _) = explainer.explain(onehot_encoded, onehot_encoded, m=10) 

    prototypes[f]={}
    prototypes[f]['W']= W
    prototypes[f]['S']= S
    prototypes[f]['data'] = data
    prototypes[f]['original'] = original

### Evaluate the set of prototypical respondents from various questionaires using their income questionaire. 

How well do the prototypes of each questionaire represent the Income questionnaire based on the objective function that Protodash uses?

By ranking the different questionnaires with their objective function values in ascending order. The higher a questionaire appears in the list, the better its prototypes represent the income questionaire. The values on the right indicate our objective value where lower value is better.

In [ ]:
#load income dataset INQ_H and its prototypes
X = prototypes['INQ_H.csv']['original']
Xdata = prototypes['INQ_H.csv']['data']

# Iterate through all questionnaires and evaluate how well their prototypes represent the income dataset. 
objs = []
for i in range(len(nhanes_files)):
        
    #load a dataset, its prototypes & weights

    f = nhanes_files[i]
    
    Ydata = prototypes[f]['data']
    S = prototypes[f]['S']
    W = prototypes[f]['W']
    
    
    # sort the order of prototypes in set S
    idx = np.argsort(S)
    S = S[idx]
    W = W[idx]
    
    # access corresponding prototypes in X. 
    XS = X[np.isin(Xdata[:, 0], Ydata[S, 0]), :]
    
    #print(Ydata[S, 0])
    #print(Xdata[np.isin(Xdata[:, 0], Ydata[S, 0]), 0])   
    
    temp = np.dot(XS, np.transpose(X))    
    u = np.sum(temp, axis=1)/temp.shape[1]
    
    K = np.dot(XS, XS.T)
    
    # evaluate prototypes on income based on our objective function with dot product as similarity measure
    obj = 0.5 * np.dot(np.dot(W.T, K), W) - np.dot(W.T, u)
    objs.append(obj)    
    

# sort the objectives (ascending order) 
index = np.argsort(np.array(objs))

# load the results in a dataframe to print
evalresult = []
for i in range(0,len(index)):    
    evalresult.append([ nhanesinfo[index[i]], objs[index[i]] ])
    
    
df_evalresult = pd.DataFrame.from_records(evalresult)
df_evalresult.columns = ['Questionaire', 'Prototypes representative of Income']
df_evalresult

### Insight from Protodash

* Early childhood represents income the most. The early childhood questionnaire has information about the environment that the child was born and raised in
* This is consistent with a [long term study](https://www.theatlantic.com/business/archive/2016/07/social-mobility-america/491240/) that talks about significant decrease in social mobility in recent times, stressing the fact that your childhood impacts how monetarily successful you are likely to be
* Protodash was able to uncover this relationship with access to just these survey questionnaires

<a class="anchor" id="IRIS"></a>
# Understanding model predictions - IRIS

There are two ways to use [SHAP](https://github.com/slundberg/shap) explainers after installing aix360. In this notebook they are invoked similarly to other explainer algorithms in aix360 via the implemented wrapper classes. But since SHAP comes pre-installed in aix360, the explainers can simply be invoked directly.

<a class="anchor" id="explain-single-shap"></a>
## Explain a single prediction

An example with a K nearest neighbors ([knn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)) classification of the IRIS dataset based on the [original SHAP tutorial](https://slundberg.github.io/shap/notebooks/Iris%20classification%20with%20scikit-learn.html).

Learn more about SHAP in [this chapter](https://christophm.github.io/interpretable-ml-book/shap.html#shap-summary-plot) in the Interpretable Machine Learning by Christoph Molnar.

The goal of SHAP is to explain the prediction of an instance x by computing the contribution of each feature to the prediction. Features with large absolute Shapley values are important. 

In [ ]:
shap.datasets.iris()

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(*shap.datasets.iris(), test_size=0.2, random_state=0)

def print_accuracy(f):
    print("Accuracy = {0}%".format(100*np.sum(f(X_test) == Y_test)/len(Y_test)))
    time.sleep(0.5) # to let the print get out before any progress bars

shap.initjs()

n_neighbors = 5   # default=5
weights='uniform'  # 'uniform' or 'distance'
knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
knn.fit(X_train, Y_train)

print_accuracy(knn.predict)

In [ ]:
X_train.head()

In [ ]:
Y_train

In [ ]:
# probability estimates
knn.predict_proba(X_train)

In [ ]:
shapexplainer = KernelExplainer(knn.predict_proba, X_train)

In [ ]:
help(shapexplainer)

In [ ]:
# aix360 style for explaining input instances
shap_values = shapexplainer.explain_instance(X_test.iloc[0,:])

In [ ]:
X_test.iloc[0,:]

### The individual force plot

Red/blue: Features that push the prediction higher (to the right) are shown in red, and those pushing the prediction lower are in blue.

The plot is centered on the x-axis at explainer.expected_value. All SHAP values are relative to the model's expected value like a linear model's effects are relative to the intercept.

In [ ]:
shapexplainer.explainer.expected_value[0]

In [ ]:
shap_values[0]

In [ ]:
shap.force_plot(shapexplainer.explainer.expected_value[0], shap_values[0], X_test.iloc[0,:])

In [ ]:
X_test.iloc[23,:]

In [ ]:
shap_values = shapexplainer.explain_instance(X_test.iloc[23,:])
shap.force_plot(shapexplainer.explainer.expected_value[0], shap_values[0], X_test.iloc[23,:])

<a class="anchor" id="explain-all-shap"></a>
## Explain all predictions


In [ ]:
shap_values_all = shapexplainer.explain_instance(X_test)
shap.summary_plot(shap_values_all, X_test, plot_type="bar")

In [ ]:
# aix360 style for explaining input instances
shap_values = shapexplainer.explain_instance(X_test)
shap.force_plot(shapexplainer.explainer.expected_value[0], shap_values[0], X_test)

<a class="anchor" id="retention"></a>
# Employee Retention

The **TED_CartesianExplainer** is an implementation of the algorithm in the AIES'19 paper by [Hind et al.]() It is most suited for use cases where matching explanations to the mental model of the explanation consumer is the highest priority; i.e., where the explanations are similar to what would be produced by a domain expert.

There are many approaches to implementing this functionality. Below the **TED_CartesianExplainer** is used, which simply takes the Cartesian product of the label and explanation and creates a new label (YE) and uses this to train a (multiclass) classifier. 

### [A deck of cards](https://en.wikipedia.org/wiki/Cartesian_product)

* An illustrative example is the standard 52-card deck
* The standard playing card ranks {A, K, Q, J, 10, 9, 8, 7, 6, 5, 4, 3, 2} form a 13-element set
* The card suits {♠, ♥, ♦, ♣} form a four-element set
* The Cartesian product of these sets returns a 52-element set consisting of 52 ordered pairs, which correspond to all 52 possible playing cards.

This simple cartesian product approach is quite general in that it can use any classifier (passed as a parameter), as long as it complies with the fit/predict paradigm.

## Synthetic dataset

A synthetically generated [dataset](https://github.com/IBM/AIX360/blob/master/aix360/data/ted_data/Retention.csv) is used with this [code](https://github.com/IBM/AIX360/blob/master/aix360/data/ted_data/GenerateData.py) that is part of aix360.

### Assigning labels

25 rules are created, for why a retention action is needed to reduce the chances of an employee choosing to leave our fictitious company. These rules are motivated by common scenarios, such as not getting a promotion in a while, not being paid competitively, receiving a disappointing evaluation, being a new employee in certain organizations with inherently high attrition, not having a salary that is consistent with positive evaluations, mid-career crisis, etc.   

Each of these 25 rules would result in the label "Yes"; i.e., the employee is a risk to leave the company. Because the rules capture the reason for the "Yes", we use the rule number as the explanation (E), which is required by the TED framework.

If none of the rules are satisfied, it means the employee is not a candidate for a retention action; i.e., a "No" label is assigned.  

### Dataset characteristics

10,000 fictious employees (X) are generated and the 26 (25 Yes + 1 No) rules are applied to produce Yes/No labels (Y), using these rules as explanations (E).  After applying these rules, the resulting dataset has the following characteristics:
- Yes (33.8%)
- No (66.2%)

First create a new TEDDataset object based on the "Retention.csv" file. The load_file method decomposes the dataset into its X, Y, and E components. (See [TEDDataset class](https://github.com/IBM/AIX360/blob/master/aix360/datasets/ted_dataset.py) for the expected format.):

In [ ]:
# Decompose the dataset into X, Y, E     
X, Y, E = TEDDataset().load_file('Retention.csv')
print("X's shape:", X.shape)
print("Y's shape:", Y.shape)
print("E's shape:", E.shape)
print()

In [ ]:
X

Partition these instances into train and test sets:

In [ ]:
# set up train/test split
X_train, X_test, Y_train, Y_test, E_train, E_test = train_test_split(X, Y, E, test_size=0.20, random_state=0)
print("X_train shape:", X_train.shape, ", X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape, ", Y_test shape:", Y_test.shape)
print("E_train shape:", E_train.shape, ", E_test shape:", E_test.shape)

## Train a classifier

In [ ]:
# Create classifier and pass to TED_CartesianExplainer
estimator = svm.SVC(kernel='linear')
# estimator = DecisionTreeClassifier()
# estimator = RandomForestClassifier()
# estimator = AdaBoostClassifier()

ted = TED_CartesianExplainer(estimator)

In [ ]:
print("Training the classifier")

ted.fit(X_train, Y_train, E_train)   # train classifier

<a class="anchor" id="ted1"></a>
## Explain a single prediction

The trained TED classifier is now ready for predictions with explanations.   We construct some raw feature vectors, created from the original dataset, and ask for a label (Y) prediction and its explanation (E).

In [ ]:
# Create an instance level example 
X1 = [[1, 2, -11, -3, -2, -2,  22, 22]]

# correct answers:  Y:-10; E:13
Y1, E1 = ted.predict_explain(X1)
print("Predicting for feature vector:")
print(" ", X1[0])
print("\t\t      Predicted \tCorrect")
print("Label(Y)\t\t " + np.array2string(Y1[0]) + "\t\t   -10")
print("Explanation (E) \t " + np.array2string(E1[0]) + "\t\t   13")
print()

In [ ]:
X2 = [[3, 1, -11, -2, -2, -2, 296, 0]]

## correct answers: Y:-11, E:25
Y2, E2 = ted.predict_explain(X2)
print("Predicting for feature vector:")
print(" ", X2[0])

print("\t\t      Predicted \tCorrect")
print("Label(Y)\t\t " + np.array2string(Y2[0]) + "\t\t   -11")
print("Explanation (E) \t " + np.array2string(E2[0]) + "\t\t   25")

In [ ]:
Y2

In [ ]:
E2

### Create a more relevant human interface

TED_CaresianExplainer can produce the correct explanation for a feature vector, but simply producing "3" as an explanation is not sufficient in most uses. This section shows one way to implement the mapping of real explanations to the explanation IDs that TED requires. This is inspired by the [FICO reason codes](https://www.fico.com/en/latest-thinking/product-sheet/us-fico-score-reason-codes), which are explanations for a FICO credit score.  

In this case the explanations are text, but the same idea can be used to map explanation IDs to other formats, such as a file name containing an audio or video explanation.

In [ ]:
Label_Strings =["IS", "Approved for"]
def labelToString(label) :
    if label == -10 :
        return "IS"
    else :
        return "IS NOT"

Explanation_Strings = [
    "Seeking Higher Salary in Org 1",
    "Promotion Lag, Org 1, Position 1",
    "Promotion Lag, Org 1, Position 2",
    "Promotion Lag, Org 1, Position 3",
    "Promotion Lag, Org 2, Position 1",
    "Promotion Lag, Org 2, Position 2",
    "Promotion Lag, Org 2, Position 3",
    "Promotion Lag, Org 3, Position 1",
    "Promotion Lag, Org 3, Position 2",
    "Promotion Lag, Org 3, Position 3",
    "New employee, Org 1, Position 1",
    "New employee, Org 1, Position 2",
    "New employee, Org 1, Position 3",
    "New employee, Org 2, Position 1",
    "New employee, Org 2, Position 2",
    "Disappointing evaluation, Org 1",
    "Disappointing evaluation, Org 2",
    "Compensation does not match evaluations, Med rating",
    "Compensation does not match evaluations, High rating",
    "Compensation does not match evaluations, Org 1, Med rating",
    "Compensation does not match evaluations, Org 2, Med rating",
    "Compensation does not match evaluations, Org 1, High rating",
    "Compensation does not match evaluations, Org 2, High rating",
    "Mid-career crisis, Org 1",
    "Mid-career crisis, Org 2",
    "Did not match any retention risk rules"]


print("Employee #1 " + labelToString(Y1[0]) + " a retention risk with explanation: " + Explanation_Strings[E1[0]])
print()
print("Employee #2 " + labelToString(Y2[0]) + " a retention risk with explanation: " + Explanation_Strings[E2[0]])

<a class="anchor" id="ted2"></a>
## Overall model accuracy

How well does TED_Cartesian do in predicting all test labels (Y) and explanations (E)?

The "score" method of TED_Cartesian calculates this. The accuracy of predicting the combined YE labels could be of interest to researchers who want to better understand the inner workings of TED_Cartesian.


In [ ]:
YE_accuracy, Y_accuracy, E_accuracy = ted.score(X_test, Y_test, E_test)    # evaluate the classifier
print("Evaluating accuracy of TED-enhanced classifier on test data")
print(' Accuracy of predicting Y labels: %.2f%%' % (100*Y_accuracy))
print(' Accuracy of predicting explanations: %.2f%%' % (100*E_accuracy))
print(' Accuracy of predicting Y + explanations: %.2f%%' % (100*YE_accuracy))

* It is easy to use the TED_CartesianExplainer if you have a training dataset that contains explanations. The framework is general in that it can use any classification technique that follows the fit/predict paradigm, so that if you already have a favorite algorithm, you can use it with the TED framework.
* The main advantage of this algorithm is that the quality of the explanations produced are exactly the same quality as those that the algorithm is trained on.  Thus, if you teach (train) the system well with good training data and good explanations, you will get good explanations out in a language you should understand.
* The downside of this approach is that someone needs to create explanations. This should be straightforward when a domain expert is creating the initial training data: if they decide a loan should be rejected, they should know why, and if they do not, it may not be a good decision.
* However, this may be more of a challenge when a training dataset already exists without explanations and now someone needs to create the explanations.  The original person who did the labeling of decisions may no longer be available, so the explanations for the decisions may not be known.  In this case, we argue, the system is in a dangerous state.  Training data exists that no one understands why it is labeled in a certain way.   Asking the model to explain one of its predictions when no person can explain an instance in the training data does not seem consistent.
* Dealing with this situation is one of the open research problems that comes from the TED approach.

# Next up

* **Home Equity Line of Credit (HELOC) Dataset**: anonymized information about home equity line of credit (HELOC) applications made by real homeowners. The customers in this dataset have requested a credit line in the range of USD 5,000 - 150,000. 
* Use the information about the applicant in their credit report to predict whether they will make timely payments over a two year period. The machine learning prediction can then be used to decide whether the homeowner qualifies for a line of credit and, if so, how much credit should be extended
* The HELOC dataset and more information about it, including instructions to download, can be found [here](https://community.fico.com/s/explainable-machine-learning-challenge?tabset-3158a=2)

Go to the next notebook: `HELOC.ipynb`